# Factgrid -> Kepler.gl 
version 0.2 
* This [Notebook](https://github.com/salgo60/FactGrid/blob/master/FactGrid%20-%3E%20Kepler.gl.ipynb)  
  * the vision is to create something like [this](https://database.factgrid.de/tools/illuminati.html) in this Notebook
  * [video](https://www.youtube.com/watch?v=WfRQQJ0vO44) showing how binder is started and used


OBS right now you need to **install [Jupiter Notebook](https://jupyter.org/install.html)** on your local machine to run Kepler.gl we have not found a way of getting it work using e.g. [binder](https://mybinder.org/). The sparql -> [pandas](https://pandas.pydata.org/) part works using [binder](https://mybinder.org/) but not [Kepler.gl](https://pandas.pydata.org/).... 
* see [question asked in binder discussion about how to get this work](https://discourse.jupyter.org/t/kepler-gl-using-binder/5090)

### Misc links

* [The Illuminati Correspondence Fast Forward](https://blog.factgrid.de/archives/1695) the blogpost we should try to do in Jupyter
* [Create a Quick Web Map with Kepler.gl and Jupyter Notebook](https://spatial.blog.ryerson.ca/2019/11/18/create-quick-dashboard-with-kepler-gl-and-jupyter-notebook/)
* Video [Visualization Nights - Introduction to Kepler.gl](https://www.youtube.com/watch?v=b8wKEY4dlvg)
* [From Beautiful Maps to Actionable Insights: Introducing kepler.gl, Uber’s Open Source Geospatial ToolboxThe](https://eng.uber.com/keplergl/)
* [Kepler.GL & Jupyter Notebooks: Geospatial Data Visualization with Uber’s opensource Kepler.GL](https://towardsdatascience.com/kepler-gl-jupyter-notebooks-geospatial-data-visualization-with-ubers-opensource-kepler-gl-b1c2423d066f)
* [Introducing kepler.gl for Jupyter](https://medium.com/vis-gl/introducing-kepler-gl-for-jupyter-f72d41659fbf)
* [Learn How to Visualize Geospatial Data in Jupyter using kepler.gl](https://www.analyticsvidhya.com/blog/2020/06/learn-visualize-geospatial-data-jupyter-kepler/)  


In [3]:
import json,sys
import pandas as pd 
from SPARQLWrapper import SPARQLWrapper, JSON
endpointFactgrid_url = "https://database.factgrid.de/sparql"

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

1. Get the SPARQL

In [4]:
queryFactGrid = """SELECT ?item ?itemLabel ?date ?origin ?originLabel ?origin_latitude ?origin_longitude ?destination ?destinationLabel ?destination_latitude ?destination_longitude  WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
  ?item wdt:P2 wd:Q10671.
  ?item wdt:P95 ?origin.
  ?origin p:P48 ?origin_coordinate.
  ?origin_coordinate psv:P48 ?origin_node.
  ?origin_node wikibase:geoLatitude ?origin_latitude.
  ?origin_node wikibase:geoLongitude ?origin_longitude.
  ?item wdt:P97 wd:Q10677.
  ?item p:P28 ?recipient_statement.
  ?recipient_statement ps:P28 ?recipient.
  ?recipient_statement pq:P29 ?destination.
  ?destination p:P48 ?destination_coordinate.
  ?destination_coordinate psv:P48 ?destination_node.
  ?destination_node wikibase:geoLatitude ?destination_latitude.
  ?destination_node wikibase:geoLongitude ?destination_longitude.
  ?item wdt:P106 ?date.
}"""


2. Run the SPARQL

In [5]:
resultsFactGrid = get_sparql_dataframe(endpointFactgrid_url, queryFactGrid)


3. Check result

In [7]:
resultsFactGrid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3091 entries, 0 to 3090
Data columns (total 11 columns):
item                     3091 non-null object
itemLabel                3091 non-null object
date                     3091 non-null object
origin                   3091 non-null object
originLabel              3091 non-null object
origin_latitude          3091 non-null object
origin_longitude         3091 non-null object
destination              3091 non-null object
destinationLabel         3091 non-null object
destination_latitude     3091 non-null object
destination_longitude    3091 non-null object
dtypes: object(11)
memory usage: 265.8+ KB


4. Change data types

In [8]:
resultsFactGrid = get_sparql_dataframe(endpointFactgrid_url, queryFactGrid)
resultsFactGrid['origin_latitude'] = resultsFactGrid[['origin_latitude']].astype(float).values
resultsFactGrid['origin_longitude'] = resultsFactGrid[['origin_longitude']].astype(float).values
resultsFactGrid['destination_latitude'] = resultsFactGrid[['destination_latitude']].astype(float).values
resultsFactGrid['destination_longitude'] = resultsFactGrid[['destination_longitude']].astype(float).values
resultsFactGrid['date'] = pd.to_datetime(resultsFactGrid['date'])
resultsFactGrid.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3091 entries, 0 to 3090
Data columns (total 11 columns):
item                     3091 non-null object
itemLabel                3091 non-null object
date                     3091 non-null datetime64[ns, UTC]
origin                   3091 non-null object
originLabel              3091 non-null object
origin_latitude          3091 non-null float64
origin_longitude         3091 non-null float64
destination              3091 non-null object
destinationLabel         3091 non-null object
destination_latitude     3091 non-null float64
destination_longitude    3091 non-null float64
dtypes: datetime64[ns, UTC](1), float64(4), object(6)
memory usage: 265.8+ KB


5. Load the data to Kepler.gl    

**OBS** this part works only if you run [Jupiter Notebook](https://jupyter.org/install.html) on your local machine
* see [question in binder discussion](https://discourse.jupyter.org/t/kepler-gl-using-binder/5090) 
** maybe this **is solved** we now have just "Geometry column does not contain geometry." 

your current map configuration
* about saving the [config in Kepler.gl](https://docs.kepler.gl/docs/keplergl-jupyter#5-save-and-load-config)

In [10]:
#Show data in Kepler.gl map is nor shown in GITHUB 
import geopandas as gpd   
from keplergl import KeplerGl #importing KeplerGl
gdf = gpd.GeoDataFrame(resultsFactGrid, geometry=gpd.points_from_xy(resultsFactGrid.origin_latitude, resultsFactGrid.origin_latitude))
Illuminati1 = KeplerGl(height=600, width=800)  
Illuminati1.add_data(data=gdf, name="The Illuminati Correspondence ") 
Illuminati1 #show the map not seen when file uploaded to GITHUB 


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'The Illuminati Correspondence ': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

More about config and saving of kepler.gl
* [Save and export](https://docs.kepler.gl/docs/user-guides/k-save-and-export)

More links 
* [How to Create Eye-Catching Maps With Python and Kepler](https://medium.com/nightingale/how-to-create-eye-catching-maps-with-python-and-kepler-gl-e7e897eff8ac#fe6b-d40ec8d73bab)
* The Programming Historian today: [Introduction to Jupyter Notebooks](https://programminghistorian.org/en/lessons/jupyter-notebooks)